In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.metrics import AUC

import seaborn as sns
import matplotlib.pyplot as plt

from nltk.corpus import stopwords
import re
import nltk
from nltk.metrics.distance import edit_distance
from nltk.stem import PorterStemmer


In [ ]:
# data = pd.read_csv("RAW_interactions.csv")
data = pd.read_csv('review.csv')
data.head(3)

## Data Cleaning

In [ ]:
data = data[data['stars'] != 0]

data.shape

In [ ]:
# Create a new column 'sentiment' based on the 'stars' column
# Negative (0) for stars <= 3, Positive (1) for stars > 3
data['sentiment'] = data['stars'].apply(lambda x: 0 if x <= 3 else 1)

# Check the distribution of the new sentiment classes
print(data['sentiment'].value_counts())

In [ ]:

# Check the distribution of the new sentiment classes
sentiment_counts = data['sentiment'].value_counts()

# Create a bar graph for sentiment distribution
plt.figure(figsize=(8, 6))
sentiment_counts.plot(kind='bar', color=['red', 'green'])
plt.title("Sentiment Class Distribution")
plt.xlabel("Sentiment Class (0 = Negative, 1 = Positive)")
plt.ylabel("Number of Reviews")
plt.xticks(rotation=0)
plt.show()


In [ ]:


# def normalize_repeated_characters(word):
#     # Replace repeated characters with a single instance
#     return re.sub(r'(.)\1{2,}', r'\1', word)

# def remove_repeated_words(words):
#     # Remove consecutive repeated words
#     normalized_words = []
#     for i, word in enumerate(words):
#         if i == 0 or word != words[i - 1]:
#             normalized_words.append(word)
#     return normalized_words

# def normalize_spelling(word, base_words):
#     # Normalize spelling using Levenshtein distance
#     for base_word in base_words:
#         if edit_distance(word, base_word) <= 2:
#             return base_word
#     return word  # Return original word if no close match is found

# def PreprocessingText(text, base_words):
#     stop_words = set(stopwords.words('english'))

#     if isinstance(text, str):  # Ensure the input is a string
#         # Remove HTML tags, special characters, and digits
#         text = re.sub(r'&\w+;', ' ', text)  # Remove HTML entities
#         text = re.sub(r'<[^>]+>', '', text)  # Matches anything between < and >
#         text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
#         text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
#         text = re.sub(r'\d+', '', text)  # Remove numbers

#         # Lowercase the text
#         text = text.lower()

#         # Tokenize and remove stopwords
#         words = nltk.word_tokenize(text)
#         words = [word for word in words if word not in stop_words]

#         # Normalize repeated characters
#         words = [normalize_repeated_characters(word) for word in words]

#         # Remove repeated words
#         words = remove_repeated_words(words)

#         # Normalize spelling
#         words = [normalize_spelling(word, base_words) for word in words]

#         return " ".join(words)

#     return text


In [ ]:
def PreprocessingText(text):
    stop_words = set(stopwords.words('english'))
    stemmer = PorterStemmer()
    if isinstance(text, str):  # Ensure the input is a string
        # Remove HTML tags, special characters, and digits
        text = re.sub(r'&\w+;', ' ', text)  # Remove HTML entities
        text = re.sub(r'<[^>]+>', '', text) # Matches anything between < and >
        text = re.sub(r'\s+', ' ', text)    # Remove extra spaces
        text = re.sub(r'[^\w\s]', '', text) # Remove punctuation
        text = re.sub(r'\d+', '', text)     # Remove numbers

        # Lowercase the text
        text = text.lower()

        # Tokenize and remove stopwords
        words = nltk.word_tokenize(text)
        words = [word for word in words if word not in stop_words]
        stemmed_words = [stemmer.stem(word) for word in words]
        # removing HTML tags

        return " ".join(words)
    return text

In [ ]:

# Preprocessing function
data['cleaned_text'] = data['text'].apply(PreprocessingText)


## Removing Few Enteries from Postive class

In [ ]:

# Assuming `df` is your DataFrame and `sentiment` is the target column
positive_rows = data[data['sentiment'] == 1]  # Filter rows where sentiment is positive
negative_rows = data[data['sentiment'] == 0]  # Filter rows where sentiment is negative

positive_rows_to_keep = positive_rows.sample(n=8000, random_state=42)  # Select 7,000 rows to drop
positive_rows = positive_rows.drop(positive_rows_to_keep.index)  # Remove those 7,000 rows

balanced_df = pd.concat([positive_rows, negative_rows])  # Combine reduced positive rows with negative rows
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)  # Shuffle and reset index

balanced_df = balanced_df.dropna(subset=['cleaned_text'])



## Finding The suitable Method for Oversampling and Undersamplying

In [ ]:
from sklearn import model_selection, preprocessing, metrics, linear_model, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from imblearn.under_sampling import AllKNN
import pandas as pd, numpy, string
from nltk.tokenize import WordPunctTokenizer
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
#Remove Special Charactors
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import seaborn as sns

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

In [ ]:
#Return the f1 Score
def train_model(classifier, feature_vector_train, label, feature_vector_valid , valid_y, plotLabel = "Confusion Matrix"):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the Training labels on validation dataset
    y_train_pred = classifier.predict(feature_vector_train)  
    f1_train = metrics.f1_score(label, y_train_pred)

    # Evaluate on testing set
    y_test_pred = classifier.predict(feature_vector_valid)
    f1_test = metrics.f1_score(valid_y, y_test_pred)

    print(f"Training F1-Score: {f1_train:.4f}")
    print(f"Testing F1-Score: {f1_test:.4f}")

    cm = confusion_matrix(valid_y, y_test_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title(plotLabel)
    plt.show()

    print(cm)


     

In [8]:
text = "hello how aring you"

nltk.word_tokenize(text)

if isinstance(text, str):
    print("hello")

stemmer_obj = PorterStemmer()

stemmer_obj.stem(text)

hello


'hello how aring y'

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split





def tfidfVectorizationProcess( Xdata , DataLabels ):
    """
    Splits the data and applies TF-IDF vectorization.

    Args:
        Xdata (iterable): Text data for vectorization.
        DataLabels (iterable): Labels corresponding to the data.

    Returns:
        tuple: TF-IDF transformed training and test sets (X_train, X_test, y_train, y_test).
    """
    X_train, X_test, y_train, y_test = train_test_split( Xdata, 
                                                         DataLabels, 
                                                         test_size=0.3, 
                                                         random_state=42)

    tfidfVectorizer = TfidfVectorizer( ngram_range=(1, 3), 
                                        max_df=0.75, 
                                        min_df=10, 
                                        sublinear_tf=True)

    X_train = tfidfVectorizer.fit_transform( X_train )
    X_test = tfidfVectorizer.transform( X_test )
    print("Shape After Vectorization! ")
    print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

    return X_train, X_test, y_train, y_test


In [ ]:
# Step 1: TF-IDF Vectorization using the existing data
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 3),  max_df=0.75, min_df=10, sublinear_tf=True)  # Limit to top 5000 features for efficiency
X_tfidf = tfidf_vectorizer.fit_transform(data['cleaned_text'])
y_label = data['sentiment']

# Step 2: Train-test split using the existing data
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y_label, test_size=0.3, random_state=42)

# Display the shape of the resulting splits
X_train.shape, X_test.shape, y_train.shape, y_test.shape


In [ ]:
from imblearn.under_sampling import AllKNN
from collections import Counter
from sklearn import model_selection, preprocessing, metrics, linear_model, svm


# Apply All-KNN for undersampling
all_knn = AllKNN(allow_minority=False)
X_train_under, y_train_under = all_knn.fit_resample(X_train, y_train)

# Display the class distribution after undersampling
print("Class distribution after All-KNN undersampling:", Counter(y_train_under))


In [ ]:
from imblearn.over_sampling import SMOTE
from collections import Counter

def apply_simple_smote(X_train, y_train):
    """
    Apply SMOTE for oversampling the minority class.
    
    Parameters:
        X_train: Features of the training set (sparse or dense matrix).
        y_train: Labels of the training set.
    
    Returns:
        X_resampled: Resampled features.
        y_resampled: Resampled labels.
    """
    smote = SMOTE(sampling_strategy={0: y_train.value_counts()[1]}, random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_train, y_train)
    
    # Print class distribution after SMOTE
    print("Class distribution after SMOTE:", Counter(y_resampled))
    
    return X_resampled, y_resampled



In [ ]:
from imblearn.over_sampling import ADASYN
from collections import Counter

def apply_adasyn(X_train, y_train):
    """
    Apply ADASYN (Adaptive Synthetic Sampling) for oversampling the minority class.
    
    Parameters:
        X_train: Features of the training set (sparse or dense matrix).
        y_train: Labels of the training set.
    
    Returns:
        X_resampled: Resampled features.
        y_resampled: Resampled labels.
    """
    adasyn = ADASYN(random_state=777, sampling_strategy = 1.0)
    X_resampled, y_resampled = adasyn.fit_resample(X_train, y_train)
    
    # Print class distribution after ADASYN
    print("Class distribution after ADASYN:", Counter(y_resampled))
    
    return X_resampled, y_resampled


In [ ]:
# X_train_under , y_train_under = apply_smote_tomek(X_train, y_train)

In [ ]:
X_train_under , y_train_under = apply_simple_smote(X_train, y_train)

In [ ]:
X_train_under , y_train_under = apply_adasyn(X_train, y_train)

In [ ]:
train_model(svm.LinearSVC(), X_train_under, y_train_under, X_test, y_test)

In [ ]:
np.sum(y_test == 1), np.sum(y_test == 0)

In [ ]:
train_model(linear_model.LogisticRegression(random_state=0, solver='lbfgs', class_weight='balanced')
            , X_train_under, y_train_under, X_test, y_test, plotLabel="LR with SMOTE and TF-IDF(ngram_range=(1, 3),  max_df=0.75, min_df=10, sublinear_tf=True)")

In [ ]:
train_model(GradientBoostingClassifier(random_state=42)
            , X_train_under, y_train_under, X_test, y_test)

In [ ]:
from sklearn.model_selection import cross_val_score

cv_scores = cross_val_score(svm.LinearSVC(), X_tfidf, y_label, cv=5, scoring='f1')
print(f"Cross-Validation F1-Scores: {cv_scores}")
print(f"Mean F1-Score: {cv_scores.mean():.4f}")


## Logistic Regression

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

# Step 1: Oversample the data
def oversample_data(balanced_df):
    # Separate features and labels
    X = balanced_df['text']
    y = balanced_df['sentiment']
    
    # TF-IDF requires numerical input, so we oversample first
    oversampler = RandomOverSampler(sampling_strategy={0: 7000}, random_state=42)
    X_resampled, y_resampled = oversampler.fit_resample(X.to_frame(), y)  # Use `.to_frame()` for X
    print("Class distribution after oversampling:", Counter(y_resampled))
    return X_resampled['text'], y_resampled

# Apply oversampling
X_resampled, y_resampled = oversample_data(balanced_df)

# Step 2: TF-IDF Feature Extraction
tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # Limit to 5000 features
X_tfidf = tfidf_vectorizer.fit_transform(X_resampled)

# Step 3: Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y_resampled, test_size=0.3, random_state=42)

xsx

In [ ]:

# Step 4: Train Logistic Regression Model
logistic_model = LogisticRegression(random_state=42)
logistic_model.fit(X_train, y_train)


In [ ]:
# Step 5: Predictions and F1 Score for Training Data
y_pred_train = logistic_model.predict(X_train)
f1_train = f1_score(y_train, y_pred_train, average='weighted')
print(f"Logistic Regression F1 Score (Train): {f1_train}")

# Step 6: Predictions and F1 Score for Test Data
y_pred_test = logistic_model.predict(X_test)
f1_test = f1_score(y_test, y_pred_test, average='weighted')
print(f"Logistic Regression F1 Score (Test): {f1_test}")

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

# Step 2: Confusion Matrix
cm = confusion_matrix(y_test, y_pred_test)

# Display Confusion Matrix
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=[0, 1], yticklabels=[0, 1])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

# Step 3: Precision, Recall, F1-Score Breakdown
report = classification_report(y_test, y_pred_test, target_names=["Class 0", "Class 1"])
print("Classification Report:")
print(report)


## Gradient Boosting

In [ ]:

# Step 5: Train Gradient Boosting Model
gb_model = GradientBoostingClassifier(random_state=42)
gb_model.fit(X_train, y_train)

In [ ]:
# Step 5: Predictions and F1 Score for Training Data
y_pred_train = gb_model.predict(X_train)
f1_train = f1_score(y_train, y_pred_train, average='weighted')
print(f"Logistic Regression F1 Score (Train): {f1_train}")

# Step 6: Predictions and F1 Score for Test Data
y_pred_test = gb_model.predict(X_test)
f1_test = f1_score(y_test, y_pred_test, average='weighted')
print(f"Logistic Regression F1 Score (Test): {f1_test}")

## Prediction

In [ ]:

# Function to predict sentiment
def predict_sentiment(text, vectorizer, model):
    # Preprocess the text
    processed_text = PreprocessingText(text)
    # Transform the text using the trained TF-IDF vectorizer
    text_tfidf = vectorizer.transform([processed_text])
    # Predict using the trained Logistic Regression model
    prediction = model.predict(text_tfidf)
    prediction_proba = model.predict_proba(text_tfidf)
    return prediction[0], prediction_proba

# Example text input
new_text = "Very very very good that it smells like old"

# Predict sentiment for the new text
predicted_label, predicted_proba = predict_sentiment(new_text, tfidf_vectorizer, logistic_model)

# Output the result
print(f"Predicted Sentiment: {'Positive' if predicted_label == 1 else 'Negative'}")
print(f"Prediction Probabilities: {predicted_proba}")


## LSTM Model

In [ ]:
# Re-import necessary libraries after reset
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

In [ ]:
# Define LSTM model
def build_lstm_model(input_dim, output_dim, input_length):
    """
    Build a simple LSTM model.

    Parameters:
    input_dim (int): Size of the vocabulary.
    output_dim (int): Dimension of the embedding layer.
    input_length (int): Length of the input sequences.

    Returns:
    model (Sequential): Compiled LSTM model.
    """
    model = Sequential([
        Embedding(input_dim=input_dim, output_dim=output_dim, input_length=input_length),  # Embedding layer
        LSTM(64, return_sequences=False),  # LSTM layer with 64 units
        Dropout(0.2),  # Dropout for regularization
        Dense(1, activation='sigmoid')  # Output layer for binary classification
    ])

    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=[tf.keras.metrics.AUC(name='auc')]
    )
    return model


In [ ]:

# Hyperparameters
vocab_size = 5000  # Vocabulary size for embedding layer
embedding_dim = 64  # Embedding vector size
max_sequence_length = 100  # Assume fixed max sequence length for input


# Build and compile the model
lstm_model = build_lstm_model(vocab_size, embedding_dim, max_sequence_length)

# Train the model
history = lstm_model.fit(
    X_train, y_train,  # Training data
    epochs=10,  # Number of epochs
    batch_size=32,  # Batch size
    validation_data=(X_test, y_test)  # Validation set
)

# Plot Training and Validation AUC
plt.figure(figsize=(12, 6))
plt.plot(history.history['auc'], label='Training AUC')
plt.plot(history.history['val_auc'], label='Validation AUC')
plt.title('Training and Validation AUC')
plt.xlabel('Epochs')
plt.ylabel('AUC')
plt.legend()
plt.show()

# Plot Training and Validation Loss
plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Plot AUC and Loss Curves
def plot_training_curves(history):
    # Extract data
    auc_train = history.history['auc']
    auc_val = history.history['val_auc']
    loss_train = history.history['loss']
    loss_val = history.history['val_loss']
    epochs = range(1, len(auc_train) + 1)
    
    # Create subplots
    plt.figure(figsize=(12, 5))
    
    # AUC Plot
    plt.subplot(1, 2, 1)
    plt.plot(epochs, auc_train, label='Train AUC')
    plt.plot(epochs, auc_val, label='Validation AUC')
    plt.title('AUC Curve')
    plt.xlabel('Epochs')
    plt.ylabel('AUC')
    plt.legend()
    
    # Loss Plot
    plt.subplot(1, 2, 2)
    plt.plot(epochs, loss_train, label='Train Loss')
    plt.plot(epochs, loss_val, label='Validation Loss')
    plt.title('Loss Curve')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

# Call the function to plot curves
plot_training_curves(history)


# **Final CLear Project**